In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.999677,False,33,1,38.0
1,6.768191,False,22,1,36.0
2,7.625790,False,32,1,41.0
3,6.563162,True,30,1,40.0
4,6.563162,False,35,1,37.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9990.000000,10000.000000,10000.000000,9873.000000
mean,7.272743,27.275300,1.038600,38.673149
std,1.338334,6.153153,0.204241,2.622336
min,0.617294,13.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.110807,32.000000,1.000000,40.000000
max,11.437582,51.000000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5172
False    4828
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
5690,0,37,1,39.0
9887,1,19,1,40.0
7224,1,19,1,38.0
6421,1,14,1,41.0
500,0,33,1,40.0


In [8]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [11]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
209/209 [==============================] - 1s 6ms/step - loss: 1.4740 - mae: 0.9429 - mse: 1.4740 - val_loss: 1.2761 - val_mae: 0.8922 - val_mse: 1.2761
Epoch 2/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3965 - mae: 0.9275 - mse: 1.3965 - val_loss: 2.6029 - val_mae: 1.3271 - val_mse: 2.6029
Epoch 3/10
209/209 [==============================] - 1s 5ms/step - loss: 1.3588 - mae: 0.9117 - mse: 1.3588 - val_loss: 2.1403 - val_mae: 1.2158 - val_mse: 2.1403
Epoch 4/10
209/209 [==============================] - 1s 5ms/step - loss: 1.3087 - mae: 0.8984 - mse: 1.3087 - val_loss: 1.1339 - val_mae: 0.8390 - val_mse: 1.1339
Epoch 5/10
209/209 [==============================] - 1s 4ms/step - loss: 1.2879 - mae: 0.8891 - mse: 1.2879 - val_loss: 1.3654 - val_mae: 0.9303 - val_mse: 1.3654
Epoch 6/10
209/209 [==============================] - 1s 5ms/step - loss: 1.2606 - mae: 0.8781 - mse: 1.2606 - val_loss: 1.1153 - val_mae: 0.8254 - val_mse: 1.1153
Epoch 7/10
209/2

In [12]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [13]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.9281864
Actual val:  6.5477291814

Predicted val:  7.8601804
Actual val:  7.87491199864

Predicted val:  8.876429
Actual val:  7.06140625186

Predicted val:  7.398431
Actual val:  5.2690480617999995

Predicted val:  8.192033
Actual val:  8.811876612139999

Predicted val:  8.200438
Actual val:  7.12534030784

Predicted val:  8.097127
Actual val:  7.3854857769999995

Predicted val:  7.7213817
Actual val:  6.33167616464

Predicted val:  7.9281864
Actual val:  7.81318256528

Predicted val:  8.090048
Actual val:  8.50102482272



In [14]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [15]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [16]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…

In [17]:
NEW NEW NEW '09-10-2020'

SyntaxError: invalid syntax (<ipython-input-17-f33c0b421fe1>, line 1)

In [18]:
read

NameError: name 'read' is not defined